In [43]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [1]:
def get_soup_from_url(url):
    """
    This function takes a URL as an argument, makes an HTTP request to the URL,
    and returns a BeautifulSoup object containing the parsed HTML content of the page.
    
    :param url: URL of the website to fetch and parse
    :return: BeautifulSoup object
    """
    # Send an HTTP GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code != 200:
        raise Exception(f"Error while fetching the URL: {url}. Status code: {response.status_code}")

    # Get the content of the response
    page_content = response.content

    # Create a BeautifulSoup object and specify the parser
    soup = BeautifulSoup(page_content, 'html.parser')

    return soup

In [40]:
def get_lyrics(lyrics_url):
    soup = get_soup_from_url(lyrics_url)
    lyrics_tag = soup.select_one('body > div.container.mt-4.mb-5 > main > div.col-md-8 > article')
    paragraphs = lyrics_tag.find_all('p')
    stanzas = []
    
    for paragraph in paragraphs:
        for br in paragraph.find_all('br'):
            br.replace_with('\n')
        stanzas.append(paragraph.text)
    
    lyrics = '\n\n'.join(stanzas)
    return lyrics
print(get_lyrics('https://korol-i-shut.su/songs/text/smelchak-i-veter.html'))

Припев:

Я ведь не из робких,
Все мне по плечу.
Сильный я и ловкий,
Ветра проучу!

Дул сильный ветер, крыши рвал.
И, несмотря на поздний час,
В округе вряд ли кто-то спал -
Стихия не на шутку разошлась.

Но вдруг какой-то парень с криком побежал
И принялся махать метлой:
"Ах, ветер, негодяй, ты спать мне помешал,
А ну-ка выходи на бой!"

(Припев)

И ветер закружился, заметался
И ели начал с корнем рвать:
"Откуда этот сумасшедший взялся,
Что хочет с ветром воевать".

Но парень не сдавался и метлой махал,
И удалялся вглубь полей.
И впрямь неплохо с ветром воевал,
А ветер становился злей...

(Припев)

Но вдруг метла со свистом улетела прочь
И храбрый парень вслед за ней.
А после этого спокойней стала ночь -
Исчез во мраке дуралей.

Его под утро пастухи нашли в стогу -
Он очень крепко спал,
А ветер песни напевал ему
И кудри ласково трепал.

(Припев)  


In [56]:
def get_song_dicts_from_album(album_url, album_id):
    soup = get_soup_from_url(album_url)
    parent_element = soup.select_one('body > div.container.mt-4.mb-5 > main > div.col-md-8')
    if parent_element is None:
        raise Exception("Нельзя найти тег, в котором содержатся песни")
    songs_h2 = soup.find_all('h2')
    song_dicts = []
    for song_h2 in songs_h2:
        song_a = song_h2.find('a')
        if song_a:
            text = song_a.text
            song_url = song_a.get('href')
            
            song_name = ' '.join(text.split())
            lyrics = get_lyrics(song_url)
        else:
            text = song_h2.text
            # remove song serial number
            splited = text.split()
            song_name = ' '.join(splited[1:])
            lyrics = ''
            
        song_dict = {'title': song_name,
                     'lyrics': lyrics,
                     'album_id': album_id}
        song_dicts.append(song_dict)
    return song_dicts 

In [57]:
def get_album_name_year(album_text):
    parts = album_text.split()
    album_name_words = []
    album_year = None
    for part in parts:
        is_year = re.search(r'^\((\d+)\)$',  part)
        if is_year:
            album_year = int(is_year.groups()[0])
        else:
            album_name_words.append(part)
    album_name = ' '.join(album_name_words)
    return album_name, album_year

get_album_name_year('\nКамнем по Голове\n                (1996)\n            ')

('Камнем по Голове', 1996)

In [59]:
error_h2s = []

def get_album_dicts():
    
    soup = get_soup_from_url('https://korol-i-shut.su/albums/')
    parent_element = soup.select_one('body > div.container.mt-4.mb-5 > main > div')
    if not parent_element:
        raise Exception('Альбомы не найдены ')
    
    album_dicts = []
    h2_tags = parent_element.find_all('h2')
    # Iterate over each 'h2' tag
    for album_id, album_h2 in enumerate(h2_tags):
        try:
            album_text = album_h2.text
            album_name, album_year = get_album_name_year(album_text)
            album_a = album_h2.find('a')
            album_url = album_a.get('href')
            song_dicts = get_song_dicts_from_album(album_url, album_id)
            album_dict = {
                'title': album_name,
                'year': album_year,
                'album_id': album_id, 
                'songs': song_dicts
            }
            album_dicts.append(album_dict)
        except Exception as e:
            error_h2s.append({
                'h2': album_h2,
                'error': e
            })
    return  album_dicts


In [60]:
album_dicts = get_album_dicts()

In [62]:
from copy import deepcopy
songs_arr_for_df = []
albums_arr_for_df = []
for album_dict in album_dicts:
    album_dict = deepcopy(album_dict)
    songs_arr_for_df.extend(album_dict['songs'])
    del album_dict['songs']
    albums_arr_for_df.append(album_dict)
    
songs_df = pd.DataFrame(songs_arr_for_df)
albums_df = pd.DataFrame(albums_arr_for_df)

In [63]:
songs_df

,title,lyrics,album_id
0,Смельчак и Ветер,"Припев:\n\nЯ ведь не из робких,\nВсе мне по пл...",0
1,Проказник Скоморох,"На свадьбе скоморох,\nБыл прытким как горох.\n...",0
2,Верная Жена,"Дождливой ночью парень, выбравшись из леса\nВд...",0
3,Садовник,"Целый вечер нет покоя парню от его сестер:\n""Т...",0
4,Блуждают Тени,"Какой таинственной казалась мне та ночь,\nЯ за...",0
...,...,...,...
307,Проклятый старый дом,В заросшем парке\nСтоит старинный дом.\nЗабиты...,17
308,Прыгну со скалы,"С головы сорвал ветер мой колпак,\nЯ хотел люб...",17
309,Внезапная голова,"Услыхал мужик под вечер вдруг,\nВ свою дверь к...",17
310,Мёртвый анархист,Ослепший старый маг\nНочью по лесу бродил.\nНа...,17


In [64]:
albums_df

,title,year,album_id
0,Камнем по Голове,1996,0
1,Король и Шут,1996,1
2,Акустический альбом,1999,2
3,Ели Мясо Мужики,1999,3
4,Герои и Злодеи,2000,4
5,Собрание,2000,5
6,Как в старой сказке,2001,6
7,"Жаль, нет ружья!",2002,7
8,Мёртвый Анархист,2003,8
9,Бунт на корабле,2004,9


In [71]:

albums_df.to_csv('data/kish_albums.csv', index=False)

In [72]:
songs_df.to_csv('data/kish_songs.csv', index=False, header=True)

In [73]:
pd.read_csv('data/kish_songs.csv')

,title,lyrics,album_id
0,Смельчак и Ветер,"Припев:\n\nЯ ведь не из робких,\nВсе мне по пл...",0
1,Проказник Скоморох,"На свадьбе скоморох,\nБыл прытким как горох.\n...",0
2,Верная Жена,"Дождливой ночью парень, выбравшись из леса\nВд...",0
3,Садовник,"Целый вечер нет покоя парню от его сестер:\n""Т...",0
4,Блуждают Тени,"Какой таинственной казалась мне та ночь,\nЯ за...",0
...,...,...,...
307,Проклятый старый дом,В заросшем парке\nСтоит старинный дом.\nЗабиты...,17
308,Прыгну со скалы,"С головы сорвал ветер мой колпак,\nЯ хотел люб...",17
309,Внезапная голова,"Услыхал мужик под вечер вдруг,\nВ свою дверь к...",17
310,Мёртвый анархист,Ослепший старый маг\nНочью по лесу бродил.\nНа...,17


In [77]:
res = re.match(r'o', 'Hello world')
print(res)

None


In [82]:
res = re.search(r'o', 'Hello world')
print(res)

<re.Match object; span=(4, 5), match='o'>


In [102]:
re.findall('(o+)(w+)', 'oowwkow')

[('oo', 'ww'), ('o', 'w')]

In [104]:
re.findall('(o+)(w+)', 'oowwkow')

[('oo', 'ww'), ('o', 'w')]

In [84]:
re.findall('(?P<os>o+)(?P<ws>w+)', 'oowwkow')

[('oo', 'www'), ('o', 'w')]

In [103]:
re.match('(?P<os>o+)(?P<ws>w+)', 'oowwkow').groupdict()

{'os': 'oo', 'ws': 'ww'}

In [98]:
re.search('(?P<os>o+)(?P<ws>w+)', 'oowwwkkooww').groupdict()

{'os': 'oo', 'ws': 'www'}

In [79]:
res = re.findall(r'o', 'Hello world')
res

['o', 'o']